In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [ ]:
#tail -n +2 train.csv | split -l 150000


In [ ]:
train = pd.read_csv('train_splits/xaa', float_precision='round_trip', header=None)
train.columns = ['acoustic_data','time_to_failure']

In [ ]:
with pd.option_context('display.precision', 15):
    print(train.head())
    print(train.tail())

In [ ]:
all_features = ['acoustic_data', 'rolling_100']

In [ ]:
features = all_features

In [ ]:
train.info(memory_usage='deep')

In [ ]:
from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train_splits'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [ ]:
splits

In [ ]:
splits_chunk = splits[0:101]

In [ ]:
training = []
ys = []

from sklearn.preprocessing import StandardScaler,MinMaxScaler 

for chunk in splits_chunk:
    path = 'train_splits/%s' % (chunk)
    chunk_df = pd.read_csv(path, float_precision='round_trip', header=None)
    chunk_df.columns = ['acoustic_data','time_to_failure']
    chunk_df['rolling_100'] = chunk_df['acoustic_data'].rolling(window=100, min_periods=1).mean()#.apply(np.log)
    chunk_df[all_features] = StandardScaler().fit_transform(chunk_df[all_features])
    training.append(chunk_df[all_features].values)
    ys.append(chunk_df['time_to_failure'].values[-1])

training = np.array(training)
ys = np.array(ys)

In [ ]:
ys.shape

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data, y_train, y_val = train_test_split(training, ys, test_size=0.1, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


dropout=0.2

my_model = Sequential()

my_model.add(LSTM(use_bias = True,unit_forget_bias=True,units = 3,\
                  dropout=dropout,recurrent_dropout=dropout,input_shape = (training.shape[1],len(features))))
my_model.add(Dense(1))

my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()




callbacks = [
    EarlyStopping(monitor='val_loss', patience=0, verbose=0)
]

history = my_model.fit(train_data, y_train, batch_size=3, epochs=100,
                      validation_data=(val_data,y_val), callbacks=callbacks
                      )


import matplotlib.pyplot as plt


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

import math
print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))